In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F

In [2]:
class GATLayer(nn.Module):
    """
    Implementation of Graph Attentiona Layer
    """
    
    def __init__(self):
        super(GATLayer, self).__init__()
        
    def forward(self, x, adj):
        print("")

In [3]:
in_features = 5
out_features = 2
nb_nodes = 3

W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
nn.init.xavier_uniform_(W.data, gain=1.414)

x = torch.rand(nb_nodes, in_features)

h = torch.mm(x, W)
N = h.size()[0]

print(h.shape)

torch.Size([3, 2])


In [4]:
a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
nn.init.xavier_uniform_(a.data, gain=1.414)

print(a.shape)

leakyrelu = nn.LeakyReLU(0.2)

torch.Size([4, 1])


In [5]:
a_input = torch.cat(
    [h.repeat(1, N).view(N*N, -1), h.repeat(N, 1)],
    dim=1).view(N, -1, 2*out_features)

In [6]:
e = leakyrelu(torch.matmul(a_input, a).squeeze(2))

In [7]:
print(a_input.shape)
print(a.shape)
print(torch.matmul(a_input, a).shape)
print(torch.matmul(a_input, a).squeeze(2).shape)

torch.Size([3, 3, 4])
torch.Size([4, 1])
torch.Size([3, 3, 1])
torch.Size([3, 3])


In [8]:
e

tensor([[ 0.1153,  0.0884, -0.0582],
        [-0.0075, -0.0129, -0.0888],
        [ 0.4446,  0.4177,  0.0381]], grad_fn=<LeakyReluBackward0>)

## masked attention

In [10]:
adj = torch.randint(2, (3,3))

zero_vec = -915*torch.ones_like(e)

print(zero_vec.shape)

torch.Size([3, 3])


In [11]:
attention = torch.where(adj>0, e, zero_vec)
print(adj, "\n", e, "\n", zero_vec)
attention

tensor([[1, 1, 1],
        [1, 1, 1],
        [0, 1, 1]]) 
 tensor([[ 0.1153,  0.0884, -0.0582],
        [-0.0075, -0.0129, -0.0888],
        [ 0.4446,  0.4177,  0.0381]], grad_fn=<LeakyReluBackward0>) 
 tensor([[-915., -915., -915.],
        [-915., -915., -915.],
        [-915., -915., -915.]])


tensor([[ 1.1530e-01,  8.8434e-02, -5.8241e-02],
        [-7.5102e-03, -1.2884e-02, -8.8812e-02],
        [-9.1500e+02,  4.1773e-01,  3.8096e-02]], grad_fn=<SWhereBackward>)

In [13]:
attention = torch.nn.functional.softmax(attention, dim=1)
h_prime = torch.matmul(attention, h)

In [14]:
attention

tensor([[0.3553, 0.3459, 0.2987],
        [0.3429, 0.3410, 0.3161],
        [0.0000, 0.5938, 0.4062]], grad_fn=<SoftmaxBackward>)

In [15]:
h_prime

tensor([[-0.0400, -0.0061],
        [-0.0502, -0.0220],
        [ 0.0858,  0.0867]], grad_fn=<MmBackward>)

In [16]:
h

tensor([[-0.1569, -0.0287],
        [ 0.5218,  0.6579],
        [-0.5515, -0.7481]], grad_fn=<MmBackward>)

## Building the Layer